In [27]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 29, Finished, Available, Finished)

In [28]:
from pyspark.sql import functions as f

 
 

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 30, Finished, Available, Finished)

In [29]:
fact = spark.table("silver_transactions").alias("f")
dim  = spark.table("dim_customer").alias("d")


StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 31, Finished, Available, Finished)

In [30]:
df_enriched = fact.join(
    dim,
    (f.col("f.customer_id") == f.col("d.customer_id")) &
    (f.col("f.transaction_date") >= f.col("d.effective_startdate")) &
    (f.col("f.transaction_date") <  f.col("d.effective_enddate")),
    "left"
)


StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 32, Finished, Available, Finished)

In [31]:
display(df_enriched)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bed47da7-ba85-4d8b-8e97-8333b2f13f8c)

In [32]:
df_selelct_enriched=(
    df_enriched
    .select(f.col("f.transaction_id"),
      f.col("f.transaction_date"),
      f.col("d.customer_sk"),
      f.col("f.amount"),
      f.col("d.risk_rating")
)
)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 34, Finished, Available, Finished)

In [33]:
display(df_selelct_enriched)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 35, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 61c1f078-f66d-430d-b59d-3e835fa67e54)

In [34]:
df_add_columns =(
    df_selelct_enriched.withColumn(
        "risk_band",
    f.when(f.col("risk_rating")=="HIGH","R3")
    .when(f.col("risk_rating")=="MEDIUM","R2")
    .otherwise("R1")
    )
)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 36, Finished, Available, Finished)

In [35]:
display(df_add_columns)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 988def7a-2f04-4b92-ad94-0e502adae783)

In [36]:
df_add_trans=(
    df_add_columns
    .withColumn(
   "transaction_amount_bucket",
   f.when(f.col("amount") >=1000,"large_amount")
   .when(f.col("amount")>= 500 ,"medium_amount")
   .otherwise("small_amount")
    )
)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 38, Finished, Available, Finished)

In [38]:
from pyspark.sql.functions import expr

df_final=(
    df_add_trans
    .withColumn(
        "high_risk_transactions",
        f.when(
            expr("risk_band ='R3'  and transaction_amount_bucket ='large_amount' "),
            1
            ).otherwise(0)
    )
    )


StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 40, Finished, Available, Finished)

In [39]:
display(df_final)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 333fb285-6020-4dda-8905-88ebe8cba48f)

In [40]:
(
    df_final
    .write
    .mode("append")
    .format("delta")
    .saveAsTable("fact_transactions_enriched")
)

StatementMeta(, e5dab885-4726-45aa-87f1-ae63788d95b2, 42, Finished, Available, Finished)